# Product mix

## Overview

In this problem, consider a set of resources $I$ used to produce a set of products $J$. Each product demands predefined ammounts of each resource. Profit margins per unit of each product are known.

Let us denote $c_{j}$ the margin associated to each product $j \in J$ and $b_{i}$ each reactant $i \in I$ availability. The ammount of $i$ used per unit of $j$ is given by $f_{i, j}$. Our objective is to maximize operational profit margin.

$$
\begin{align}
    \text{max} \quad & \sum_{j \in J} c_j x_j \\
    \text{s.t.} \quad & \sum_{j \in J} f_{i, j} x_{j} \leq b_{i} & \forall \; i \in I \\
    & x_{j} \geq 0 & \forall \; j \in J \\
\end{align}
$$

## Example

In this example, we consider six chemical species:
- Reactants {A, B, C}
- Products {D, E, F}.

Table 1: Reactants
| Reactant | Availability (lb/day) |
| --- | --- |
| A | 8000 |
| B | 3000 |
| C | 2500 |

 
Table 2: Products
| Product | Unitary margin ($/lb) |
| --- | --- |
| D | 2.15 |
| E | 1.34 |
| F | 1.72 |

Table 3: Proportions
| Reactant \ Product | D | E | F
| --- | --- | --- | --- |
| A | 7/10 | 1/3 | 1/2 |
| B | 1/5 | 2/3 | 1/6 |
| C | 1/10 | 0 | 1/3 |


Our goal is to maximize daily profit.

Let us denote $c_{j}$ the margin associated to each product $j \in J$ and $b_{i}$ each reactant $i \in I$ availability. The ammount of $i$ used per unit of $j$ is given by $f_{i, j}$

In [1]:
import numpy as np
from scipy.optimize import linprog
import pyomo.environ as pyo

## Scipy

In [2]:
# Pseudo costs
margins = np.array([2.15, 1.34, 1.72])
c = - margins

# A matrix
A = np.array([
    [7/10, 1/3, 1/2],
    [1/5, 2/3, 1/6],
    [1/10, 0.0, 1/3]
])

b = np.array([8000.0, 3000.0, 2500.0])

In [3]:
sol = linprog(c, A_ub=A, b_ub=b)
print(sol.x)

[7105.26315789 1026.31578947 5368.42105263]


## Pyomo

In [4]:
# Initialize model
model = pyo.ConcreteModel()

# Sets of reactants and products
model.I = pyo.Set(initialize=["A", "B", "C"])
model.J = pyo.Set(initialize=["D", "E", "F"])

# Availability
model.b = pyo.Param(model.I, initialize=dict(zip(model.I, b)))

# Margins (now as a maximization objective)
model.c = pyo.Param(model.J, initialize=dict(zip(model.J, margins)))

# Proportions
proportions = {}
for k, i in enumerate(model.I):
    for l, j in enumerate(model.J):
        proportions[i, j] = A[k, l]

model.f = pyo.Param(model.I, model.J, initialize=proportions)

# Decision variables
model.x = pyo.Var(model.J, within=pyo.NonNegativeReals, name="x")

In [5]:
# Availability constraint
def availability_rule_cstr(model, i):
    return sum(model.f[i, j] * model.x[j] for j in model.J) <= model.b[i]

model.cstr_available = pyo.Constraint(model.I, rule=availability_rule_cstr, name="cstr_available")

In [6]:
# Objective function
def obj_func(model):
    return sum(model.x[j] * model.c[j] for j in model.J)

model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

In [7]:
cbc = pyo.SolverFactory("cbc")
sol = cbc.solve(model, tee=False)

In [8]:
model.x.display()

x : Size=3, Index=J
    Key : Lower : Value     : Upper : Fixed : Stale : Domain
      D :     0 : 7105.2632 :  None : False : False : NonNegativeReals
      E :     0 : 1026.3158 :  None : False : False : NonNegativeReals
      F :     0 : 5368.4211 :  None : False : False : NonNegativeReals


In [9]:
model.cstr_available.display()

cstr_available : Size=3
    Key : Lower : Body               : Upper
      A :  None :  8000.000056666666 : 8000.0
      B :  None : 3000.0000233333335 : 3000.0
      C :  None :         2500.00002 : 2500.0


In [10]:
model.obj.display()

obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 25885.263344
